# Customer Experience Data Analizi

## 1. Veri Seti Seçimi
Bu analizde kullanılan veri seti: **Customer Experience Data**
Veri seti, müşteri etkileşimleri, satın alım ve davranış verileri ile memnuniyet ve elde tutma durumlarını içermektedir.

## Gerekli kütüphaneler

In [ ]:
import pandas as pd  # Veri analizi için pandas
import numpy as np  # Sayısal işlemler için numpy
import matplotlib.pyplot as plt  # Grafik çizimi için matplotlib
import seaborn as sns  # Gelişmiş görselleştirme için seaborn

# Ayar: Grafik stili beyaz ızgara olarak belirleniyor

In [ ]:
sns.set(style="whitegrid")

## 1. Veri Setini Yükleme

In [ ]:
print("Veri seti yükleniyor...")
df = pd.read_csv("customer_experience_data.csv")  # CSV dosyasından veri yükleniyor
print("Veri seti başarıyla yüklendi. İlk 5 gözlem:")
print(df.head())  # Veri setinin ilk 5 satırı gösteriliyor


## 2. Veri Seti Genel Bilgisi

In [ ]:
# Sayısal değişkenlerin istatistiksel özeti
print("\n--- Veri Seti Genel Bilgisi ---")
print(f"Toplam gözlem sayısı: {df.shape[0]}")
print(f"Toplam değişken sayısı: {df.shape[1]}")
print("Veri tipleri:")
print(df.dtypes.value_counts())  # Veri tiplerinin dağılımı yazdırılıyor

# Sayısal ve kategorik sütunların ayrılması

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
if "Customer_ID" in numeric_cols:
    numeric_cols.remove("Customer_ID")  # Kimlik sütunu analiz dışı bırakılıyor

## 3. Sayısal Değişkenlerin Özeti

In [ ]:
print("\n--- Sayısal Değişkenlerin Özeti ---")
describe_df = df[numeric_cols].describe().T[['mean', 'std', 'min', '50%', 'max']]
describe_df.columns = ['Ortalama', 'Standart Sapma', 'Minimum', 'Medyan', 'Maksimum']
print(describe_df)  # Sayısal özet tablo yazdırılıyor

## 4. Sayısal Değişkenlerin Dağılımı (Histogramlar)

In [ ]:
print("\n--- Sayısal Değişkenlerin Dağılımı ---")
df[numeric_cols].hist(bins=20, figsize=(15, 10), color="skyblue", edgecolor="black")
plt.suptitle("Sayısal Değişkenlerin Dağılımı", fontsize=16)
plt.tight_layout()
plt.show()

## 5. Kategorik Değişkenlerin Özeti

In [ ]:
print("\n--- Kategorik Değişkenlerin Özeti ---")
for col in categorical_cols:
    print(f"{col}: benzersiz değer sayısı = {df[col].nunique()}, en sık değer = {df[col].mode()[0]}")


## 6. Kategorik Değişkenlerin Dağılımı (Barplot)

In [ ]:
print("\n--- Kategorik Değişkenlerin Dağılımı ---")
for col in categorical_cols:
    plt.figure(figsize=(6, 4))
    sns.countplot(data=df, x=col, order=df[col].value_counts().index)
    plt.title(f"{col} Dağılımı")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 7. Eksik Değer Analizi

In [ ]:
print("\n--- Eksik Değer Analizi ---")
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
if not missing.empty:
    print("Eksik veri içeren sütunlar ve sayıları:")
    print(missing)
    # Eksik veri oranı barplot ile gösterilir
    missing_percent = (missing / len(df)) * 100
    plt.figure(figsize=(8, 4))
    sns.barplot(x=missing_percent.values, y=missing_percent.index, palette="Reds_r")
    plt.xlabel("Eksik Veri Oranı (%)")
    plt.title("Eksik Verilerin Görselleştirilmesi")
    plt.tight_layout()
    plt.show()
else:
    print("Veri setinde eksik veri bulunmamaktadır.")


## 8. Aykırı Değer Analizi (IQR Yöntemi)

In [ ]:
def detect_outliers_iqr(data, column):
    q1 = data[column].quantile(0.25)
    q3 = data[column].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return data[(data[column] < lower) | (data[column] > upper)]

for col in numeric_cols:
    outliers = detect_outliers_iqr(df, col)
    oran = len(outliers) / len(df) * 100
    print(f"{col} sütununda {len(outliers)} aykırı değer bulundu (%{oran:.2f}).")

# 9. Boxplot ile Aykırı Değer Görselleştirme

In [ ]:
print("\n--- Boxplot ile Aykırı Değerlerin Görselleştirilmesi ---")
for col in numeric_cols:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=df[col], color="lightblue")
    plt.title(f"{col} - Boxplot")
    plt.tight_layout()
    plt.show()

print("\nAnaliz tamamlandı.")